## Estimate the stan model

In [ ]:
import pandas as pd
import numpy as np
import cmdstanpy
from cmdstanpy import CmdStanModel
import arviz
import seaborn as sns
import sklearn
from sklearn import preprocessing
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing
import ast
import os, shutil

#### Choose which model specification to run

In [ ]:
model_type='beta'  # change this to run the robustness tests with different specifications
stan_path='stan_specification/'

if model_type=='nonlinear':
    model = CmdStanModel(stan_file=stan_path+'nonlinear.stan')
    seed = 1
    output_folder='nonlinear/'
    coefficient_dict={'intercept':'beta1','sndi':'beta2','density':'beta3','precip':'beta4','min_temp':'beta5',
                    'bikelanes':'beta6','slope':'beta7','includes_inboundoutbound':'beta8','rail_in_city':'beta9','max_temp':'beta10','population':'beta11',
                    'min_temp2':'beta12','max_temp2':'beta13','density2':'beta14'}


if model_type in['linear','high_gdp','beta','trips']:
    if model_type=='linear':
        model = CmdStanModel(stan_file=stan_path+'linear.stan')
    else:
        model = CmdStanModel(stan_file=stan_path+'beta.stan')
    seed = 1 
    output_folder=model_type+'/'
    coefficient_dict={'intercept':'beta1','sndi':'beta2','density':'beta3','precip':'beta4','min_temp':'beta5',
                      'bikelanes':'beta6','slope':'beta7','includes_inboundoutbound':'beta8','rail_in_city':'beta9','max_temp':'beta10','population':'beta11',
                      'min_temp2':'beta12'}

if model_type=='inoutbound_only':
    model = CmdStanModel(stan_file=stan_path+'inoutbound_only.stan') #  same as beta with one less variable
    seed = 2 # for seed 1, chains diverge
    output_folder='inoutbound_only/'
    coefficient_dict={'intercept':'beta1','sndi':'beta2','density':'beta3','precip':'beta4','min_temp':'beta5',
                      'bikelanes':'beta6','slope':'beta7','rail_in_city':'beta8','max_temp':'beta9','population':'beta10',
                      'min_temp2':'beta11'} # same as beta but drop includes_inboundoutbound
    
if model_type== 'saturated':
    model = CmdStanModel(stan_file=stan_path+'saturated.stan')
    seed = 1
    output_folder='saturated/'
    coefficient_dict={'intercept':'beta1','sndi':'beta2','density':'beta3','precip':'beta4','min_temp':'beta5',
                'bikelanes':'beta6','slope':'beta7','includes_inboundoutbound':'beta8','rail_in_city':'beta9','max_temp':'beta10',
                'sndi_added':'beta11','population':'beta12','motorways':'beta13'}


In [ ]:
# specify dependent variables and column names in the dataframe
if model_type=='trips':
    y_bike_name='tripshare_cycling_23'
    y_walk_name='tripshare_on_foot_23' 
else:
    y_bike_name='km_share_cycling_no_transit'
    y_walk_name='km_share_on_foot_no_transit'

# calculate the number of city-level variables
num_city_x=len(coefficient_dict.keys())-1
city_var_order={key: index for index, key in enumerate(coefficient_dict.keys())}

# Create arrays of city-level variables
city_data=pd.read_csv('data/data_23.csv')

# drop state column first, because we don't use it, and it has a lot of missing data
city_data = city_data.drop('state',axis=1).dropna()

# Calculate country-level means, in order to do marginal effects
vlist = [vv+'_standard' for vv in coefficient_dict.keys() if vv+'_standard' in city_data.columns]
country_means = city_data.groupby('country_num')[vlist].mean()

# add squared values
for col in coefficient_dict:
    if col.endswith('2'):
        country_means[col+'_standard'] = country_means[col[:-1]+'_standard'] **2
# add binary variables
country_means = country_means.join(city_data.groupby('country_num')[['includes_inboundoutbound','rail_in_city']].mean())

country_means.sort_index(inplace=True) # sort by country_num, which is the index

if model_type=='inoutbound_only':
    city_data = city_data[city_data.includes_inboundoutbound==1].reset_index()
if model_type=='high_gdp':
    gdp_cutoff = city_data.groupby('country_num').gdp_standard.mean().median()  # mean to get country-level GDP, then take median
    city_data = city_data[city_data.gdp_standard>=gdp_cutoff].reset_index()

# dependent variables
y_bike = city_data[y_bike_name]
y_walk = city_data[y_walk_name]

# independent variables
country_num = city_data.country_num.values
sndi_standard = city_data.sndi_standard.values
density_standard = city_data.density_standard.values
density_standard2 = city_data.density_standard2.values
precip_standard = city_data.precip_standard.values
min_temp_standard = city_data.min_temp_standard.values
min_temp_standard2 = city_data.min_temp_standard2.values
max_temp_standard = city_data.max_temp_standard.values
max_temp_standard2 = city_data.max_temp_standard2.values
bikelanes_standard = city_data.bikelanes_standard.values
slope_standard = city_data.slope_standard.values
sndi_added=city_data.sndi_added_standard.values
population=city_data.population_standard.values
includes_inboundoutbound=city_data.includes_inboundoutbound.values
motorways_standard=city_data.motorways_standard.values
rail_in_city=city_data.rail_in_city.values
n_country = city_data.groupby('country_num')['feature_id'].count()

# create arrays from country-level variables
country_data=pd.read_csv('data/country_23.csv')
gdp_standard = country_data.gdp_standard.values
dependency_standard = country_data.dependency_standard.values
gasoline_standard=country_data.gasoline_standard.values
country_num_alpha=country_data.index.values

if model_type!='linear':
    # beta model can't accept values of 0 so we make all of these 0.0001
    city_data['bike_share2']=city_data[y_bike_name].apply(lambda x: x if x>0 else 0.0001)
    y_bike=city_data.bike_share2.values


In [ ]:
hierarchical_data = {'N': len(y_bike),
                          'J': len(gdp_standard),
                          'L' : num_city_x,
                          'M' : 3,   
                          'country_num': country_num+1, # Stan counts starting at 1
                          'gdp_percapita': gdp_standard,
                          'dependency_ratio':dependency_standard,
                          'gas_prices':gasoline_standard,
                          'sndi': sndi_standard,
                          'density': density_standard,
                          'density2': density_standard2,
                          'precip': precip_standard,
                          'min_temp': min_temp_standard,
                          'min_temp2':min_temp_standard2,
                          'max_temp': max_temp_standard,
                          'max_temp2': max_temp_standard2,
                          'bikelanes':bikelanes_standard,
                          'slope':slope_standard,
                          'sndi_added':sndi_added,
                          'population':population,
                          'includes_inboundoutbound':includes_inboundoutbound.astype(int),
                          'motorways':motorways_standard,
                          'rail_in_city':rail_in_city.astype(int),
                          'y_bike': y_bike,
                          'y_walk': y_walk,
                          'bikelanes_global_mean':city_data.bikelane_per_road_km.mean(),
                          'bikelanes_global_stdev':np.std(city_data.bikelane_per_road_km)}

# add group-level means
for col in country_means.columns:
        hierarchical_data[col.replace('_standard','')+'_mean'] = country_means[col].values
      

# read in fit or fit the model
# change path to where you want the (very large) stan output files to reside
outputpath = '../stan_output/'
if not(os.path.exists(outputpath)): os.mkdir(outputpath)
    
# if you want to read in previous estimates
#combined_fit=cmdstanpy.from_csv(path=outputpath, method=None)

# if you want to run the model. this will take a few hours
combined_fit = model.sample(data=hierarchical_data, inits=0.5, seed=seed, iter_warmup=1000, iter_sampling=1000, max_treedepth=15, adapt_delta=0.9,output_dir=outputpath)


#### Basic Fit

In [ ]:
print(combined_fit.diagnose())

In [ ]:
# see https://mc-stan.org/docs/2_27/stan-users-guide/posterior-p-values.html
fig, axes = plt.subplots(1,2, figsize=(7, 3))
for mode, yvar, ax in zip(['walk','bike'],[y_walk,y_bike], axes):
    forarviz = arviz.from_cmdstanpy(
        posterior=combined_fit,
        predictions = 'y_hat_'+mode,
        posterior_predictive='y_rep_'+mode, 
        observed_data={"y": yvar})


    print('% of samples for {} that are greater than mean: {:.1f}'.format(mode, 100*forarviz.posterior['mean_gt_'+mode].mean().values))
    print('Should be about 50%')
    arviz.plot_bpv(forarviz, kind="p_value", data_pairs = {'y' : 'y_rep_'+mode}) 
    arviz.plot_ppc(forarviz, data_pairs = {'y' : 'y_rep_'+mode}, ax=ax) 
    ax.set_xlabel('{} mode share'.format(mode.title()), size=11)
    ax.set_xlim(-0.1,0.3)
    ax.legend(loc=1, prop={'size': 9})
    ax.set_xticks([0,0.1, 0.2, 0.3])
    ax.tick_params(axis='x', which='major', labelsize=10)

    vnames = ['tau_'+mode,'gamma_'+mode] +['phi_'+mode]*('beta' in model_type)
    arviz.plot_trace(forarviz, var_names=vnames)

fig.tight_layout()
if not(os.path.exists(outputpath)): os.mkdir(outputpath)
fig.savefig(outputpath+'ppc_{}.png'.format(model_type), dpi=300)
    

In [ ]:
# get the summary for all variables
combined_summary=combined_fit.summary(percentiles=(5, 50, 95), sig_figs=6)
combined_summary.reset_index(names=['var_name'],inplace=True)
def extract_values(row):
    return ast.literal_eval(row)[0], ast.literal_eval(row)[1]

In [ ]:
yhats=pd.DataFrame()
for mode in ['walk', 'bike']:
    # filter to just estimates of betas (city-level predictors)
    yhat_summary=combined_summary[combined_summary.var_name.str.contains(f'y_hat_{mode}')].copy()
    yhat_summary.var_name = yhat_summary.var_name.apply(lambda x: x.replace('mu_gen','y_hat'))
    # unpack the city
    yhat_summary.var_name=yhat_summary.var_name.str.replace(f'y_hat_{mode}','')
    yhat_summary['city']=yhat_summary['var_name'].apply(lambda x: ast.literal_eval(x)[0]-1)
    yhat_summary.reset_index(drop=True)
    yhat_summary.rename(columns={'Mean':f'y_hat_mean_{mode}'},inplace=True)
    yhats=pd.concat([yhats,yhat_summary.set_index('city')[[f'y_hat_mean_{mode}']]],axis=1)

In [ ]:
# create prediction df with y_hats and difference between actual and expected modeshare
prediction_df=city_data.join(yhats)
prediction_df.plot(kind='scatter',y='y_hat_mean_bike',x=y_bike_name,s=0.5)
prediction_df.plot(kind='scatter',y='y_hat_mean_walk',x=y_walk_name,s=0.5)

prediction_df.to_csv(outputpath+'cmdstan_'+model_type+'_prediction_df.csv')

In [ ]:
# calculate mean squared error
plt.close('all')
MSE_walk=sklearn.metrics.mean_squared_error(prediction_df[y_walk_name], prediction_df['y_hat_mean_walk'])
MSE_bike=sklearn.metrics.mean_squared_error(prediction_df[y_bike_name], prediction_df['y_hat_mean_bike'])
pd.DataFrame([MSE_walk,MSE_bike],columns=['MSE'],index=['walk', 'bike']).to_csv(outputpath+'MSE_'+model_type+'.csv')
print(MSE_walk,MSE_bike)

#### Trace plots (for convergence check)

In [ ]:
# reformat fit.stan_variables()['beta'] to a set of betas that are plottable in arviz
betas = {}

betas['betas_bike']=combined_fit.stan_variables()['beta_bike']
betas['betas_walk']=combined_fit.stan_variables()['beta_walk']

beta1_bike=[]
beta2_bike=[]
beta3_bike=[]
beta4_bike=[]
beta5_bike=[]
beta6_bike=[]
beta7_bike=[]
beta8_bike=[]
beta9_bike=[]
beta10_bike=[]
beta11_bike=[]
beta12_bike=[]
beta13_bike=[]
beta14_bike=[]
beta15_bike=[]
beta1_walk=[]
beta2_walk=[]
beta3_walk=[]
beta4_walk=[]
beta5_walk=[]
beta6_walk=[]
beta7_walk=[]
beta8_walk=[]
beta9_walk=[]
beta10_walk=[]
beta11_walk=[]
beta12_walk=[]
beta13_walk=[]
beta14_walk=[]
beta15_walk=[]

# loop through the city-level variables and create arrays of all coefficient estimates
for mode in ['bike','walk']:
    for var in range(1,len(coefficient_dict)+1):
        betas[f'beta{var}_{mode}']=np.asarray(betas[f'betas_{mode}'])[:,var-1,:].T

# now these into dictionaries with the underlying variable name
for mode in ['bike','walk']:
    for key, value in coefficient_dict.items():
        betas[f'{value}_{mode}']={f'{key}_{mode}':betas[f'{value}_{mode}']}
        
for mode in ['bike','walk']:
    for value in coefficient_dict.values():
        arviz.plot_trace(data=betas[f'{value}_{mode}'],chain_prop='color',figsize=(10, 4))
        plt.tight_layout()
        plt.savefig(outputpath+'betas_{}_{}_{}.png'.format(value, mode, model_type), dpi=300)

In [ ]:
plt.close('all')

In [ ]:
import matplotlib.pyplot as plt
gammas = {}
gammas['gammas_bike'] = combined_fit.stan_variables()['gamma_bike']
gammas['gammas_walk'] = combined_fit.stan_variables()['gamma_walk']

intercept_bike=[]
sndi_bike=[]
density_bike=[]
density2_bike=[]
precip_bike=[]
min_temp_bike=[]
min_temp2_bike=[]
max_temp_bike=[]
max_temp2_bike=[]
bikelanes_bike=[]
slope_bike=[]
includes_inboundoutbound_bike=[]
rail_in_city_bike=[]
sndi_added_bike=[]
population_bike=[]
motorways_bike=[]

intercept_walk=[]
sndi_walk=[]
density_walk=[]
density2_walk=[]
precip_walk=[]
min_temp_walk=[]
min_temp2_walk=[]
max_temp_walk=[]
max_temp2_walk=[]
bikelanes_walk=[]
slope_walk=[]
includes_inboundoutbound_walk=[]
rail_in_city_walk=[]
sndi_added_walk=[]
population_walk=[]
motorways_walk=[]

coeff_labels = ['Intercept','GDP per capita', 'Dependency ratio', 'Gasoline price']

for mode in ['bike','walk']:
    for city_var, i in city_var_order.items():
        gammas[f'{city_var}_{mode}']=np.asarray(gammas[f'gammas_{mode}'])[:,i,:].T
    
# now do the coefficient plots!
for mode in ['bike','walk']:
    for key in city_var_order.keys():
        ax=arviz.plot_trace(data=gammas[f'{key}_{mode}'],chain_prop='color',legend=True,figsize=(10, 4))
        ax[0,0].legend(labels=['Intercept','','GDP per capita','','Dependency ratio','','Gasoline price'], fontsize=7, loc='upper right')
        ax[0,0].set_title(f'{mode}_{key}')
        ax[0,1].set_title(f'{mode}_{key}')
        fig = plt.gcf()
        fig.savefig(outputpath+'countrycoeffs_{}_{}_{}.pdf'.format(key, mode, model_type))
            

#### Extracting Estimates for Further Analysis

In [ ]:
beta_estimates=pd.DataFrame()

# for beta model, we save marginal effects. for linear model, the actual beta coefficients
cname = 'beta' if model_type=='linear' else 'me'
for mode in ['bike','walk']:
    # filter to just estimates of betas (city-level predictors)
    betas_mode_summary=combined_summary[combined_summary.var_name.str.contains(f'{cname}_{mode}')].copy()
    # unpack the country and variable
    betas_mode_summary.var_name=betas_mode_summary.var_name.str.replace(f'{cname}_{mode}','')
    if cname=='me':  # we need to average the marginal effect over country
        n_vars = len(coefficient_dict)
        betas_mode_summary[['country', 'beta']] = betas_mode_summary['var_name'].apply(lambda x: pd.Series(extract_values(x)))
        betas_mode_summary['country']=np.repeat(city_data.country_num.values, n_vars)
        betas_mode_summary = betas_mode_summary.groupby(['country','beta'])[['5%','50%','95%','Mean']].mean().reset_index()
    else:
        assert model_type=='linear'
        betas_mode_summary[['beta','country']] = betas_mode_summary['var_name'].apply(lambda x: pd.Series(extract_values(x)))
        # combined effect of linear and squared for min temp
        me = combined_summary[combined_summary.var_name.str.contains(f'me_{mode}_mintemp')]
        me['country'] = city_data.country_num.values + 1
        me = me.groupby('country')[['5%','50%','95%','Mean']].mean().reset_index()
        me['beta'] = 12  # variable number
        betas_mode_summary = pd.concat([betas_mode_summary[betas_mode_summary.beta!=12], me])
    betas_mode_summary['beta']=betas_mode_summary['beta'].apply(lambda x: 'beta'+str(x)+'_'+mode)
    # reformat to be one row per city instead of one row per city-variable
    beta_mode_pivot=pd.pivot_table(data=betas_mode_summary,values=['5%','50%','95%','Mean'],index='country',columns='beta',aggfunc='mean',fill_value=None)
    new_columns = ['_'.join(map(str, reversed(col))) for col in beta_mode_pivot.columns]
    beta_mode_pivot.columns=new_columns
    beta_estimates=pd.concat([beta_estimates,beta_mode_pivot],axis=1)
# replace country number with country name, 0-indexed
beta_estimates=beta_estimates.reset_index().drop(columns=['country']).join(country_data.country).set_index('country')
beta_estimates.to_csv(outputpath+'beta_estimates_'+model_type+'.csv')

In [ ]:
gamma_estimates=pd.DataFrame()

cname = 'gamma' if model_type=='linear' else 'me_country'

for mode in ['bike','walk']:
    # filter to just estimates of gammas (country-level predictors)
    gammas_mode_summary=combined_summary[combined_summary.var_name.str.contains(f'{cname}_{mode}')].copy()
    # unpack the country and variable
    gammas_mode_summary.var_name=gammas_mode_summary.var_name.str.replace(f'{cname}_{mode}','')
    if cname=='me_country': # collapse to mean
        gammas_mode_summary[['country_var', 'city']] = gammas_mode_summary['var_name'].apply(lambda x: pd.Series(extract_values(x)))
        gammas_mode_summary.country_var=gammas_mode_summary.country_var.replace({1:'intercept',2:'gdp_per_capita',3:'dependency_ratio',4:'gas_prices'})+'_'+mode
        gammas_mode_summary = gammas_mode_summary.groupby(['country_var'])[['5%','50%','95%','Mean']].mean()
        gamma_estimates=pd.concat([gamma_estimates,gammas_mode_summary],axis=0)
    else:
        gammas_mode_summary[['city_var', 'country_var']] = gammas_mode_summary['var_name'].apply(lambda x: pd.Series(extract_values(x)))
        gammas_mode_summary['city_var']=gammas_mode_summary['city_var'].apply(lambda x: 'beta'+str(x)+'_'+mode)   
        gammas_mode_summary.country_var=gammas_mode_summary.country_var.replace({1:'intercept',2:'gdp_per_capita',3:'dependency_ratio',4:'gas_prices'})
        gamma_mode_pivot=pd.pivot_table(data=gammas_mode_summary,values=['5%','50%','95%','Mean'],index='city_var',columns='country_var',aggfunc='mean',fill_value=None)
        new_columns = ['_'.join(map(str, reversed(col))) for col in gamma_mode_pivot.columns]                                
        gamma_mode_pivot.columns=new_columns
        gamma_estimates=pd.concat([gamma_estimates,gamma_mode_pivot],axis=0)

gamma_estimates=gamma_estimates.reset_index()
gamma_estimates.to_csv(outputpath+'gamma_estimates_'+model_type+'.csv')

In [ ]:
# save summary coefficients to a table, for inclusion in the SI, with column names
median_row = int((len(beta_estimates)-1)/2) # assumes an odd number of countries
outputdf = pd.DataFrame()
if model_type=='linear':
    gamma_estimates.set_index('city_var', inplace=True) # for easier accessing
else:
    gamma_estimates.set_index('country_var', inplace=True) # for easier accessing

assert len(beta_estimates.columns)%4==0
for mode in ['walk','bike']:
    output_table = {}
    for coeff in coefficient_dict:
        colname = coefficient_dict[coeff]+'_'+mode+'_50%'
        beta_estimates.sort_values(by=colname, inplace=True)
        median_txt = '{:.3f}'.format(beta_estimates.iloc[median_row][colname])
        UIcols = colname.replace('50%','5%'),colname.replace('50%','95%') 
        UI_txt = ' ({:.3f}, {:.3f})'.format(beta_estimates.iloc[median_row][UIcols[0]], beta_estimates.iloc[median_row][UIcols[1]])
        coeffname = 'beta intercept' if coeff == 'intercept' else coeff
        output_table[coeffname] = median_txt + UI_txt
    
    # add gamma coefficients, but only for intercept (not the interactions)
    for coeff in ['dependency_ratio','gas_prices','gdp_per_capita', 'intercept']:
        if model_type=='linear':
            median_txt = '{:.3f}'.format(gamma_estimates.loc['beta1_'+mode,coeff+'_50%'])
            UI_txt = ' ({:.3f}, {:.3f})'.format(gamma_estimates.loc['beta1_'+mode,coeff+'_5%'], gamma_estimates.loc['beta1_'+mode,coeff+'_95%']) 
        else:
            median_txt = '{:.3f}'.format(gamma_estimates.loc[coeff+'_'+mode,'50%'])
            UI_txt = ' ({:.3f}, {:.3f})'.format(gamma_estimates.loc[coeff+'_'+mode,'5%'], gamma_estimates.loc[coeff+'_'+mode,'95%'])
        
        coeffname = 'gamma intercept' if coeff == 'intercept' else coeff
        output_table[coeffname] = median_txt + UI_txt
        
    outputdf[mode] = pd.Series(output_table)
    
print(outputdf)
outputdf.to_csv(outputpath+'Coefficient_table_{}.csv'.format(model_type))

### Bike lane scenarios

Here, we use the fitted model to generate estimates for new scenarios with different quantities of bicycle lanes

In [ ]:
assert model_type in ['beta','linear'] # stop here if not
tmpdir = '../tmpfns/'
if not(os.path.exists(tmpdir)): os.mkdir(tmpdir)
    
# run generated quantities for displacement estimates
# https://mc-stan.org/cmdstanpy/users-guide/examples/Run%20Generated%20Quantities.html
model_gq = CmdStanModel(stan_file=stan_path+model_type+'_gq.stan')
try: # use fit object
    gq = model_gq.generate_quantities(data=hierarchical_data, previous_fit=combined_fit, gq_output_dir=tmpdir)
except: # load in previous estimates
    csv_files = ['stan_output/'+model_type+'/'+fn for fn in os.listdir('stan_output/'+model_type) if fn.endswith('.csv')]
    print(csv_files)
    assert len(csv_files)==4
    gq = model_gq.generate_quantities(data=hierarchical_data, previous_fit=csv_files, gq_output_dir=tmpdir)

In [ ]:
# the problem is that the output files are massive (10GB per chain), and can't easily be read into memory
# so let's read in line by line and write to temporary files, which we can then read in and calculate the median
assert model_type in ['beta','linear'] # stop here if not
   
output_fns = ['tmp_gq_{}_{}'.format(model_type, ii) for ii in range(1,301)]
output_files = [open(tmpdir+fn,'w') for fn in output_fns]
gq_fns = [fn for fn in os.listdir(tmpdir) if fn.startswith(model_type+'_gq') and fn.endswith('.csv')]
assert len(gq_fns)==4

# read in all 4 chains, and write to 100 different files
headerrow = '',''
for chain, gq_fn in enumerate(gq_fns):
    print('Processing chain {} and file {}'.format(chain+1, gq_fn))
    infile = open(tmpdir+gq_fn,'r')
    # get past header
    while 'aas_w.' not in headerrow:
        headerrow = infile.readline()
    # check file structure - aas should start at column 3
    headerrow = headerrow.split(',')
    assert 'aas' not in headerrow[0]+headerrow[1] and np.all(['aas' in r for r in headerrow[2:]])
    n_cities = int(headerrow[-1].split('.')[1])
    assert n_cities==11587
    assert len(headerrow)==n_cities*3*100+2 # 3 variables, 100 scenarios per city, 2 extra cols

    # check that modes are in right order
    for i, m in enumerate(['w','b','c']):
        startidx, stopidx = 2+n_cities*100*i, 2+n_cities*100*(i+1)
        assert all([ii[:5] == 'aas_'+m for ii in headerrow[startidx:stopidx]])
    
    counter = 0
    while infile:
        row = infile.readline()
        if row=='': break # end of data
        row = row.split(',')
        counter+=1
        for i, outfile in enumerate(output_files):
            line_to_write = row[2+i*n_cities:2+(i+1)*n_cities] 
            assert len(line_to_write)==n_cities
            outfile.write(','.join(line_to_write)+'\n')
    infile.close()
    assert counter==1000 # we had 1000 sampling iterations per chain
    print('Processed {} rows from chain {}'.format(counter, chain+1))
for f in output_files: f.close()
                

In [ ]:
# load in city data and process it
# we need this to join to the simulations to calculate CO2 savings
assert model_type in ['beta','linear']
exclude_transit = False
fn_suffix='_excltransit' if exclude_transit else ''  # save files under different name
    
city_data=pd.read_csv('data/data_23.csv', index_col=0).reset_index()
if exclude_transit:
    # don't include transit in displaced mode share. The easiest way to do this is to set transit mode share to zero
    city_data.km_transit=0
    
efs = pd.read_csv('data/transport.csv')
ef_col=[col for col in efs.columns.to_list() if '_emissions_factor_kg_co2e_per_liter' in col or 'fuel_efficiency_km_per_liter' in col]
efs=efs[['feature_id','year']+ef_col].copy()
efs = efs[efs.year==2023]
city_data=city_data.set_index('feature_id').join(efs.set_index('feature_id')).reset_index()

for mode in ['automobile','motorcycle','bus','ferry','rail','subway','tram','on_foot','cycling']:
    city_data[f'{mode}_occupancy'] = 9.9 if mode=='bus' else 1.7 if mode=='automobile' else 1.0

# impute missing data by replacing with country values
for mode in ['automobile','motorcycle','bus','ferry','rail','subway','tram']:
    for col in ['_emissions_factor_kg_co2e_per_liter', '_fuel_efficiency_km_per_liter']:
        # https://stackoverflow.com/questions/19966018/filling-missing-values-by-mean-in-each-group
        city_data.loc[city_data[mode+col]==0, mode+col] = np.nan
        city_data[mode+col] = city_data.groupby('country')[mode+col].transform(lambda x: x.fillna(x.mean()))

# a few missing for bus, so replace with the global mean
bus_mean = city_data.bus_fuel_efficiency_km_per_liter.mean()
city_data.loc[city_data.km_bus>0] = city_data.loc[city_data.km_bus>0].fillna(bus_mean)

# impute remaining missing data (all MC) by using the global proportion of MC fuel economy to car
fe_ratio = city_data.groupby('country').automobile_fuel_efficiency_km_per_liter.mean().mean() / city_data.groupby('country').motorcycle_fuel_efficiency_km_per_liter.mean().mean()
city_data.loc[pd.isnull(city_data.motorcycle_fuel_efficiency_km_per_liter), 'motorcycle_fuel_efficiency_km_per_liter'] = city_data.automobile_fuel_efficiency_km_per_liter / fe_ratio 

# calculate to emissions per km, and displaced emissions per km of walk/bike
# note that emissions are already converted to PASSENGER KM 
for mode in ['automobile','motorcycle']:
    city_data[mode+'_kg_co2_e_per_km'] = city_data[mode+'_emissions_factor_kg_co2e_per_liter'] / city_data[mode+'_fuel_efficiency_km_per_liter'] / city_data[mode+'_occupancy']
    city_data[mode+'_km_share_fordisplacement'] = city_data['km_'+mode] / (city_data.km_automobile + city_data.km_motorcycle + city_data.km_transit)

# for transit, assume global average for rail - 22.35 kgCO2 / pkm 
# https://www.iea.org/energy-system/transport/rail
city_data['transit_kg_co2_e_per_km'] = (
    city_data['bus_emissions_factor_kg_co2e_per_liter'] / city_data['bus_fuel_efficiency_km_per_liter'] / city_data['bus_occupancy']
         * (city_data.km_bus / city_data.km_transit)
  +  0.02235 * (1- city_data.km_bus / city_data.km_transit))
city_data['transit_km_share_fordisplacement'] = city_data.km_transit / (city_data.km_automobile + city_data.km_motorcycle + city_data.km_transit)

# check imputation worked. this allows us to use fillna a couple of lines below
assert np.all(pd.notnull(city_data[city_data.motorcycle_km_share_fordisplacement>0]).motorcycle_kg_co2_e_per_km)

city_data['kg_co2_displaced_per_active_km'] = city_data.automobile_km_share_fordisplacement * city_data.automobile_kg_co2_e_per_km + (city_data.motorcycle_km_share_fordisplacement * city_data.motorcycle_kg_co2_e_per_km).fillna(0)
city_data['kg_co2_displaced_per_active_km_including_transit_emissions'] = (city_data.kg_co2_displaced_per_active_km + 
    (city_data.transit_km_share_fordisplacement * city_data.transit_kg_co2_e_per_km).fillna(0))
city_data['kg_co2_auto_mc'] = city_data.km_automobile * city_data.automobile_kg_co2_e_per_km + (city_data.km_motorcycle * city_data.motorcycle_kg_co2_e_per_km).fillna(0)
city_data['kg_co2_auto_mc_transit'] = city_data['kg_co2_auto_mc'] + (city_data.km_transit * city_data.transit_kg_co2_e_per_km).fillna(0)

assert np.all(pd.notnull(city_data['kg_co2_displaced_per_active_km']))
assert np.all(pd.notnull(city_data['kg_co2_displaced_per_active_km_including_transit_emissions']))
assert np.all(pd.notnull(city_data['kg_co2_auto_mc']))
assert np.all(pd.notnull(city_data['kg_co2_auto_mc_transit']))

In [ ]:
# now, read all the files in and calculate percentiles
# each file has all the data for one simulation (1-100) of the bike lanes variable, 
#     for walk, bike or combined
assert model_type in ['beta','linear'] # stop here if not

colnames = [str(ii) for ii in range(1,n_cities+1)]

# store global aggregates for each simulation
agg_medians, agg_pc_025, agg_pc_975 = {}, {}, {}

for j, mode in enumerate(['_walk','_bike','_combined']):
    output_fns_subset = output_fns[j*100:(j+1)*100]
    means, medians, pc_025, pc_975  = [], [], [], []
    agg_medians[mode], agg_pc_025[mode], agg_pc_975[mode] = {}, {}, {}
    for i, fn in enumerate(output_fns_subset):
        bl = i+1 # bike lane simulated variable starts at 1
        agg_medians[mode][bl], agg_pc_025[mode][bl], agg_pc_975[mode][bl] = {}, {}, {}
        # dataframe is N iterations x J cities, i.e. 4000 x 11587
        df = pd.read_csv(tmpdir+fn, header=None, names=colnames, dtype='float32')
        means.append(df.mean())
        medians.append(df.median())
        pc_025.append(df.quantile(q=0.025, axis=0))
        pc_975.append(df.quantile(q=0.975, axis=0))
        
        # we also calculate aggregate CO2 savings here, so that we can get the uncertainty bands
        df = df.T.reset_index(drop=True) # transpose so we can join to city_data
        co2_kg = df.mul(city_data.km_no_transit, axis=0).mul(city_data.kg_co2_displaced_per_active_km, axis=0).sum()
        # also multiple by km_notransit (because that's how the model is estimated), but include transit emissions in the displacement
        co2_kg_inctransit = df.mul(city_data.km_no_transit, axis=0).mul(city_data.kg_co2_displaced_per_active_km_including_transit_emissions, axis=0).sum()
        pc_reduction = co2_kg / city_data.kg_co2_auto_mc.sum()       
        pc_reduction_inctransit = co2_kg_inctransit / city_data.kg_co2_auto_mc_transit.sum()
        added_km = df.mul(city_data.km_no_transit, axis=0).sum()
        # compute medians and percentiles for aggregates
        for dfname in ['co2_kg', 'pc_reduction', 'added_km','co2_kg_inctransit','pc_reduction_inctransit']:
            tmpdf = globals()[dfname]
            assert len(tmpdf)==4000
            agg_medians[mode][bl][dfname] = tmpdf.median()
            agg_pc_025[mode][bl][dfname]  = tmpdf.quantile(q=0.025)
            agg_pc_975[mode][bl][dfname]  = tmpdf.quantile(q=0.975)

        del df # save memory

    for fn, adf in zip(['means','medians','pc_025','pc_975'], [means, medians, pc_025, pc_975]):
        cdf = pd.concat(adf,axis=1)
        cdf.columns=[str(ii) for ii in range(1,101)]
        outfn = outputpath+'gq_'+model_type+'_'+fn+mode+fn_suffix+'.csv'
        cdf.to_csv(outfn)
        print('Saved '+outfn)

# save aggregated
for fn, adict in zip(['medians','pc_025','pc_975'], [agg_medians, agg_pc_025, agg_pc_975]):
    adf = pd.concat({k: pd.DataFrame(v).T for k, v in adict.items()}, axis=0)
    adf.index.names=['mode','bikelanes']
    outfn = outputpath+'gq_'+model_type+'_'+fn+'_aggregated'+fn_suffix+'.csv'
    adf.to_csv(outfn)
    print('Saved '+outfn)
    

In [ ]:
# now delete tmpfiles and gq output (because that's 10GB+!)
# do this after running both exclude_transit =False and =True
gq_fns = [fn for fn in os.listdir(tmpdir) if fn.startswith(model_type+'_gq') or fn=='.DS_Store'] # including the non-csv files
for fn in output_fns+gq_fns:
    os.remove(tmpdir+fn)
os.rmdir(tmpdir)

In [ ]:
if 0 and model_type=='linear':  # deprecated
    # extract just the projected additional active shares at different bikelane scenarios
    additional_active_share=combined_summary[combined_summary.var_name.str.contains('additional_active_share')].copy()
    additional_active_share.var_name=additional_active_share.var_name.str.replace('additional_active_share','')
    additional_active_share[['city', 'bikelane_scenario']] = additional_active_share['var_name'].apply(lambda x: pd.Series(extract_values(x)))
    additional_active_share_summary=pd.pivot_table(data=additional_active_share,values=['5%','50%','95%','Mean'],index='city',columns='bikelane_scenario',aggfunc='mean',fill_value=None)
    new_columns = ['bikelanes_'+'_'.join(map(str, reversed(col))) for col in additional_active_share_summary.columns]       
    additional_active_share_summary.columns=new_columns
    # replace city number with feature_id
    additional_active_share_summary=additional_active_share_summary.reset_index().join(city_data.feature_id).drop(columns=['city']).set_index('feature_id')
    additional_active_share_summary.to_csv(outputpath'additional_active_share_'+model_type+'.csv')